<a href="https://colab.research.google.com/github/a308601801/Class-Exercises/blob/master/%E3%80%90%E9%A6%96%E6%AC%A1%E8%BF%90%E8%A1%8C%EF%BC%88%E5%AE%89%E8%A3%85%EF%BC%89%E3%80%91StableDiffusionUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StableDiffusion NovelAi Google云盘安装版

第二次及以后：点击1、2、5

Credits: https://t.me/StableDiffusion_CN

https://github.com/AUTOMATIC1111/stable-diffusion-webui

https://github.com/TheLastBen/fast-stable-diffusion
StableDiffusionUI(NAILeaks)_云盘安装版release


In [ ]:
#@title 1、授权挂载Google云盘
from google.colab import drive

#@markdown 配置stable-diffusion-webui所在文件夹名称(无特殊需要建议默认)
SD_Folder_Name='sd' #@param {type:"string"}

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

drive.mount('/content/gdrive')
!mkdir -p /content/gdrive/MyDrive/$SD_Folder_Name
sd_dir="/content/gdrive/MyDrive/"+SD_Folder_Name
webui_dir=sd_dir+"/stable-diffusion-webui"

name, memory.total [MiB], memory.free [MiB]
Tesla T4, 15360 MiB, 15109 MiB
Mounted at /content/gdrive


In [ ]:
#@title 2、初始化临时运行环境【每次新开会话时执行一次】
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output
from IPython.display import HTML
from subprocess import getoutput

%cd /content/
if '3.8' in sys.version:
  with capture.capture_output() as cap:
    !wget -q -i "https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/deps.txt"
    for i in range(1,9):
      !mv "Dpnds.{i}" "Dpnds.7z.00{i}"
    !7z x -y -o/ Dpnds.7z.001
    !rm -rf /content/usr *.00*
else:
  # Install triton
  !pip install -U -q --pre triton

if not os.path.exists('/tools/node/bin/lt'):
  !npm install -g localtunnel

gpuinfo = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')
# Install xformers for python version < 3.10
if int(sys.version.split('.')[1])<10:
  !echo "$gpuinfo"
  if 'T4' in gpuinfo:
    if '3.7' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python37/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
    elif '3.8' in sys.version:
      #%pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
      pass
    elif '3.9' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python39/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl
    else:
      !echo "install xformers failed: unsupported python version."
  elif 'P100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl
  elif 'V100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl
  elif 'A100' in gpuinfo:
    if '3.8' in sys.version:
      !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/A100
      !rm -r /usr/local/lib/python3.8/dist-packages/xformers
      !7z x -y -o/usr/local/lib/python3.8/dist-packages/ /content/A100
      !rm /content/A100
  else:
    print('it seems that your GPU is not supported at the moment, install xformers failed.')
else:
  !pip install -q xformers

!echo done

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.332s
Tesla T4
done


In [ ]:
#@title 3、下载webui程序和模型文件到云盘（仅首次安装到云盘时执行！）
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ！！如果之前版本运行有问题，先在谷歌云盘里删除SD文件夹并清空回收站再尝试

#@markdown ！！勾选以下载latest版模型（7GB，请确保云盘有足够的存储空间，不勾选默认4G，其实也够用了。）：
Use_Latest_Model=False #@param {type:"boolean"}
#@markdown ！！勾选以下载NovelAi hypernetwork文件（下载速度可能很慢，新手可以不下载）：
Download_Hypernetworks=False #@param {type:"boolean"}
#@markdown 使用NovelAi参数（建议使用）
Use_NovelAi_Config=True #@param {type:"boolean"}

%cd $sd_dir
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

!mkdir -p $webui_dir/models/Stable-diffusion $webui_dir/models/hypernetworks $webui_dir/models/deepbooru
%cd $webui_dir/models/Stable-diffusion/
!echo downloading model...
with capture.capture_output() as cap:
  !apt install -y -qq aria2
  !echo '' > ./animefull-final-pruned.ckpt && rm ./animefull-final-pruned.ckpt
  !echo '' > ./animefull-latest.ckpt && rm ./animefull-latest.ckpt
  if Use_Latest_Model:
    !aria2c --summary-interval=10 -x 3 --allow-overwrite=true -Z \
      https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.ckpt
    !wget -O $webui_dir/config.yaml https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/config-latest.yaml
  else:
    !aria2c --summary-interval=10 -x 3 --allow-overwrite=true -Z \
      https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-final-pruned.ckpt
    !wget -O $webui_dir/config.yaml https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/config-pruned.yaml

if Use_NovelAi_Config:
  !wget -O $webui_dir/config.json https://raw.githubusercontent.com/PtO2/stablediffusion_webui_configs/main/config.json
  !wget -O $webui_dir/ui-config.json https://raw.githubusercontent.com/PtO2/stablediffusion_webui_configs/main/ui-config.json

# Download deepdanbooru dataset（运行后可以点击deepdanbooru下载最新版）
#!mkdir -p $webui_dir/models/deepbooru
#!wget -O /content/deepbooru.zip https://github.com/KichangKim/DeepDanbooru/releases/download/v3-20211112-sgd-e28/deepdanbooru-v3-20211112-sgd-e28.zip
#!unzip -o -d $webui_dir/models/deepbooru /content/deepbooru.zip
#%rm -f /content/deepbooru.zip

# Install VAE Weights (optional)
!wget -O $sd_dir/animevae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt

%cd /content
# Install hypernetwork （optional)
if Download_Hypernetworks:
  !curl -L https://cloudflare-ipfs.com/ipfs/bafybeiduanx2b3mcvxlwr66igcwnpfmk3nc3qgxlpwh6oq6m6pxii3f77e/_modules.tar | tar x -C $webui_dir/models/hypernetworks

!echo done

In [ ]:
#@title 3.1、安装插件、添加模型（可选）
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output
from IPython.display import HTML
from subprocess import getoutput

#@markdown 安装动画插件
deforum=False #@param {type:"boolean"}
#@markdown 安装anything等其他模型可以参考我那期快速替换模型教程 哔哩哔哩-BV1DP4y1D7Ln

#@markdown 安装图片浏览器
imgbro=False #@param {type:"boolean"}
#@markdown 安装controlnet插件，不装可惜了,注意在本条代码中去掉#的注释即可下载更多模型，默认下载canny
controlnet=True #@param {type:"boolean"}
#@markdown 如果使用Counterfeit模型，一定要勾选下载PT，效果拔群。
Counterfeit=True #@param {type:"boolean"}
#@markdown 安装lora插件，使用此类模型需要安装。
lora=False #@param {type:"boolean"}
#@markdown 安装lora韩国小姐姐模型。
loramodel=False #@param {type:"boolean"}
%cd /content/
if deforum:
  %cd /content/
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui/ /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/deforum
if imgbro:
  %cd /content/
  !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
if controlnet:
  !git clone https://github.com/Mikubill/sd-webui-controlnet /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet
  
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models/
  !wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_canny_fp16.safetensors
  #!wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_openpose_fp16.safetensors
  #!wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_scribble_fp16.safetensors
if Counterfeit:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/embeddings/
  !wget https://github.com/tuzh6/model/raw/controlnet/EasyNegative.pt
if lora:
  %cd /content/
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-additional-networks
if loramodel:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora
  !wget https://civitai.com/api/download/models/8750 -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora/koreanDollLikeness_v10.safetensors
!echo done

In [ ]:
#@title 3.2、更新webui代码（可选）
%cd $webui_dir

#@markdown 勾选以回退到特定版本（方便更新版本无法使用回退）
Rollback_Commit=True #@param {type:"boolean"}
#@markdown 复制回退版本的sha值到此处
Commit_SHA='3715ece0adce7bf7c5e9c5ab3710b2fdc3848f39' #@param {type:"string"}

%rm -f $webui_dir/.git/index.lock
!git config core.filemode false

if Rollback_Commit:
  !git fetch
  !git reset --hard $Commit_SHA
else:
  !git reset --hard master

!git pull
#@markdown sha值可在此处查看- https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master

In [ ]:
!pip uninstall tensorfolw
!conda install tensorflow

/bin/bash: conda: command not found


In [ ]:
#@title 运行stable_diffusion_webui
from IPython.utils import capture
from subprocess import getoutput
import os,sys,time,fileinput

#@markdown 启动成功后会有一个以gradio.app或loca.lt结尾的链接，点开即可访问程序。（链接也会保存在临时环境硬盘的url.txt文件中）

#@markdown 加载VAE文件：
Use_VAE=True #@param {type:"boolean"}

#@markdown 通过localtunnel内网穿透服务访问：
Use_Localtunnel_Share=False #@param {type:"boolean"}

#@markdown 启用xformers优化显存性能（T4机型已兼容，<font color="#dd0000">其他机型若无法出图请关闭</font>）
Enable_Xformers=True #@param {type:"boolean"}

#@markdown 加载tag自动补全插件
Load_Tagcomplete=False #@param {type:"boolean"}

#@markdown 自定义启动参数
User_Cmd_Args='--reinstall-xformers --no-half-vae' #@param {type:"string"}

!mkdir -p $webui_dir/cache/huggingface $webui_dir/cache/torch
%rm -rf /root/.cache/huggingface
%rm -rf /root/.cache/torch
!ln -s $webui_dir/cache/huggingface /root/.cache/
!ln -s $webui_dir/cache/torch /root/.cache/

!mkdir -p /content/repositories
%rm -rf $webui_dir/repositories
!ln -s /content/repositories $webui_dir
if not os.path.exists('/content/repositories/stable-diffusion-stability-ai'):
  with capture.capture_output() as cap:
    !git clone https://github.com/Stability-AI/stablediffusion.git /content/repositories/stable-diffusion-stability-ai
    sdhash=getoutput("grep 'stable_diffusion_commit_hash =' "+webui_dir+"/launch.py").split(',')[1].strip('" )')
    !git -C /content/repositories/stable-diffusion-stability-ai checkout $sdhash
    !sed -i 's/        logvar_t = self\.logvar\[t\]\.to(self\.device)/        logvar_t = self\.logvar\[t\.cpu()\]\.to(self\.device)/' /content/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py

%rm -f $webui_dir/javascript/tagAutocomplete.js
%rm -f $webui_dir/scripts/tag_autocomplete_helper.py
if Load_Tagcomplete:
  if not os.path.exists(webui_dir+'/extensions/tagcomplete') or not os.path.exists('/content/tagext_updated'):
    with capture.capture_output() as cap:
      !mkdir -p $webui_dir/extensions/
      if not os.path.exists(webui_dir+'/extensions/tagcomplete'):
        !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git $webui_dir/extensions/tagcomplete
      else:
        %cd $webui_dir/extensions/tagcomplete
        !git reset --hard HEAD
        !git pull
      if int(sys.version.split('.')[1])<9:
        !pip install -q pathlib3x
        !sed -i 's/from pathlib /from pathlib3x /' $webui_dir/extensions/tagcomplete/scripts/tag_autocomplete_helper.py
      !touch /content/tagext_updated
else:
  %rm -rf $webui_dir/extensions/tagcomplete

if os.path.exists(webui_dir+'/models/torch_deepdanbooru'):
  %rm -rf $webui_dir/models/deepbooru

!sed -i 's/gradio.*/gradio/' $webui_dir/requirements.txt
!sed -i 's/gradio.*/gradio/' $webui_dir/requirements_versions.txt
#!pip install -q gradio==3.9

extra_cmd=''
grdir=getoutput('pip show gradio | grep -o "/.*"')+'/gradio'
if Use_Localtunnel_Share:
  if not os.path.exists('/content/blocks.py.bak'):
    !cp $grdir/blocks.py /content/blocks.py.bak
  %cd /content/
  !nohup lt --port 7860 > url.txt 2>&1 &
  time.sleep(2)
  srvurl=getoutput("grep -o 'https[^ ]*' /content/url.txt")

  for line in fileinput.input(grdir+'/blocks.py', inplace=True):
    if line.strip().startswith('self.server_name ='):
      line = f'            self.server_name = "{srvurl[8:]}"\n'
    if line.strip().startswith('self.protocol = "https"'):
      line = '            self.protocol = "https"\n'
    if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
      line = ''
    if line.strip().startswith('else "http"'):
      line = ''
    sys.stdout.write(line)
else:
  extra_cmd=extra_cmd+' --share'
  if os.path.exists('/content/blocks.py.bak'):
    !cp /content/blocks.py.bak $grdir/blocks.py
  #!wget -q -O $grdir/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

%cd $webui_dir
# hacks to load model in colab
!git checkout -- ./modules/*.py
if os.path.exists(webui_dir+"/models/Stable-diffusion/animefull-latest.ckpt"):
  !sed -i 's@weight_load_location =.*@weight_load_location = "cuda"@' $webui_dir/modules/shared.py
  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda' if os.path.getsize(checkpoint_file) > 5500000000 else map_location@" $webui_dir/modules/sd_models.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" $webui_dir/modules/extras.py

# launch
if Enable_Xformers:
  extra_cmd=extra_cmd+" --xformers"
if Use_VAE:
  extra_cmd=extra_cmd+" --vae-path "+sd_dir+"/animevae.pt"
extra_cmd=extra_cmd+' '+User_Cmd_Args
!COMMANDLINE_ARGS="--deepdanbooru --skip-version-check --disable-safe-unpickle --gradio-debug --config config.yaml $extra_cmd" REQS_FILE="requirements.txt" python launch.py

/content/gdrive/MyDrive/sd/stable-diffusion-webui
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Commit hash: 2c1bb46c7ad5b4536f6587d327a03f0ff7811c5d
Installing xformers
Cloning Taming Transformers into repositories/taming-transformers...
Cloning K-diffusion into repositories/k-diffusion...
Cloning CodeFormer into repositories/CodeFormer...
Cloning BLIP into repositories/BLIP...
Installing requirements for Web UI
Installing requirement for sd-webui-controlnet
Installing requirement for sd-webui-controlnet (svg_input)

Launching Web UI with arguments: --deepdanbooru --skip-version-check --disable-safe-unpickle --gradio-debug --config config.yaml --share --xformers --vae-path /content/gdrive/MyDrive/sd/animevae.pt --no-half-vae
2023-02-19 00:25:37.878992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX5